##  Project: Image Classification 

In this project, I have classified images from the CIFAR-10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html). The dataset consists of airplanes, dogs, cats, and other objects. 

In this project I have used Keras a framework which supports convolutional neural network for classifying images form CIFAR-10 dataset. I have developed my onw model based on Keras and used it for training and testing.

I have used following steps in the project

* Download Keras
* Import all the modules
* Download the images from CIFAR-10 dataset
* Augment the images dataset by performing operations like rotation
* Define Keras model
* Train and test the dataset


I have run this on my laptop (Lenovo Legin Y720) which has 6 GB GPU.



## Install Keras

In [1]:
# https://keras.io/
!pip install -q keras
import keras

notebook 5.4.0 requires ipykernel, which is not installed.
jupyter 1.0.0 requires ipykernel, which is not installed.
jupyter-console 5.2.0 requires ipykernel, which is not installed.
ipywidgets 7.1.1 requires ipykernel>=4.5.1, which is not installed.
tensorflow-gpu 1.9.0 has requirement tensorboard<1.10.0,>=1.9.0, but you'll have tensorboard 1.10.0 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Import all the modules

In [2]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD

## Tensorflow configuration changes for GPU

In [3]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

## Define Hyper parameters to be used

Following hyper parameters are used:

* batch_size - Batch size
* num_classes - Number of classes (10)
* epochs - Number of epochs (250)
* l - Number of layers
* num_filter - Number of filters 
* compression
* dropout_rate


In [4]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 250
l = 22
num_filter = 16
compression = 0.5
dropout_rate = 0.25

## Download the dataset and perform one hot encoding

In this step, I have downloaded the cifar10 training and test data

Next, performed one-hot encoding on target labels train_y and test_y to get the resultant train_y, test_y which will be used for processing

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoding 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Perform Image augmentation of images

1. Peform image augmentation by rotating 45 degrees for a randomly selected 1000 images and append the augmented images
2. Perform image augmentation by rotating 45 degrees for a randomly selected 1000 images and append the augmented images

In [6]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Rotate images by 45 degrees
datagen1 = ImageDataGenerator(rotation_range=45)

# fit parameters from data
datagen1.fit(x_train)

# Get 1000 augmented images and append to the training data
for x_rotated_images_batch, y_rotated_images_batch in datagen1.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_rotated_images_batch, axis=0)
    y_train = np.append(y_train, y_rotated_images_batch, axis=0)
    break

# Horizontally flip image
datagen2 = ImageDataGenerator(horizontal_flip=True)

# fit parameters from data
datagen2.fit(x_train)  

# Get 1000 augmented horizontally flipped images and append to the training data
for x_flipped_images_batch, y_flipped_images_batch in datagen2.flow(x_train, y_train, batch_size=1000):
    x_train = np.append(x_train, x_flipped_images_batch, axis=0)
    y_train = np.append(y_train, y_flipped_images_batch, axis=0)
    break


## Create building blocks for model

1. Create a method for add_denseblock of layers where each layer has following operations:
    perform Batch Normalization, 
    Perform relu Activation
    Apply 3x3 convolution of a number of filters num_filter*compression
    Add dropout
    
    All the layers are concated to form a dense block
    
2. Create a method add_transition which does the folloiwng:
    
    perform Batch Normalization, 
    Perform relu Activation
    Apply 1x1 convolution of a number of filters num_filter*compression
    Add dropout
    Perform 2x2 Average Pooling
    
3.  Create a method output_layer which does the folloiwng:
    
    perform Batch Normalization, 
    Perform relu Activation
    Perform 2x2 Average Pooling
    Flatten
    Dense layer of 10 classes with softmax activation
    

In [7]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [8]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return AvgPooling

In [9]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

## Create Final Model


Apply  building blocks of dense_block, transtion blocks and 3x3 convolution filters to create the final model



In [10]:
input = Input(shape=(img_height, img_width, channel,))
layer1_transition = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

layer2_block = add_denseblock(layer1_transition, num_filter, dropout_rate)
layer2_transition = add_transition(layer2_block, num_filter, dropout_rate)
skip_connection1 = layer2_transition

layer3_block = add_denseblock(layer2_transition, num_filter, dropout_rate)
layer3_transition = add_transition(layer3_block, num_filter, dropout_rate)

layer4_block = add_denseblock(layer3_transition, num_filter, dropout_rate)
layer4_transition = add_transition(layer4_block, num_filter, dropout_rate)

layer5_skip_connection_block = add_denseblock(skip_connection1, num_filter, dropout_rate)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_skip_connection_block = Conv2D(int(num_filter*compression), (5,5), use_bias=False)(layer5_skip_connection_block)
layer5_block = Concatenate(axis=-1)([layer5_skip_connection_block, layer4_transition])
layer5_transition = add_transition(layer5_block, num_filter, dropout_rate)

layer6_block = add_denseblock(layer5_transition,  num_filter, dropout_rate)
output = output_layer(layer6_block)

In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

## Compile the model

In [12]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

## Callbacks For Early Stopping and Saving the Best Model

In [13]:
from keras.callbacks import Callback
class EarlyStoppingByValidationAccuracy(Callback):
    def __init__(self, monitor='val_acc', value=1.0, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        accuracy_value= logs.get(self.monitor)
       
        if accuracy_value >= self.value:
            self.model.stop_training = True
            if self.verbose == 1:
                print("Epoch %d: Threshold for early stopping has reached" % (epoch + 1))   

In [14]:
from keras.callbacks import ModelCheckpoint
callbacks_list=[]
model_save_path= "best_model-CIFAR10-monimoy-my_computer2.h5"
callbacks_list.append(EarlyStoppingByValidationAccuracy(monitor='val_acc',  value=0.9201, verbose=1))
callbacks_list.append(ModelCheckpoint(model_save_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))

## Fit the model on the train data and calculate metrics (Training Loss, Training Accuracy, Validation Losss, Validation Accuracy)

In [15]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks_list)

Train on 52000 samples, validate on 10000 samples
Epoch 1/250
52000/52000 [==============================] - 338s 7ms/step - loss: 1.9728 - acc: 0.2591 - val_loss: 2.3979 - val_acc: 0.2055

Epoch 00001: val_acc improved from -inf to 0.20550, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 2/250
52000/52000 [==============================] - 305s 6ms/step - loss: 1.6869 - acc: 0.3636 - val_loss: 1.8543 - val_acc: 0.3592

Epoch 00002: val_acc improved from 0.20550 to 0.35920, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 3/250
52000/52000 [==============================] - 307s 6ms/step - loss: 1.5632 - acc: 0.4158 - val_loss: 1.6657 - val_acc: 0.4076

Epoch 00003: val_acc improved from 0.35920 to 0.40760, saving model to best_model-CIFAR10-monimoy-my_computer2.h5
Epoch 4/250
52000/52000 [==============================] - 307s 6ms/step - loss: 1.4708 - acc: 0.4556 - val_loss: 2.1125 - val_acc: 0.3503

Epoch 00004: val_acc did not improve from 0.40760
E


Epoch 00078: val_acc did not improve from 0.79500
Epoch 79/250
52000/52000 [==============================] - 310s 6ms/step - loss: 0.5294 - acc: 0.8159 - val_loss: 0.7955 - val_acc: 0.7593

Epoch 00079: val_acc did not improve from 0.79500
Epoch 80/250
52000/52000 [==============================] - 310s 6ms/step - loss: 0.5233 - acc: 0.8164 - val_loss: 0.6713 - val_acc: 0.7888

Epoch 00080: val_acc did not improve from 0.79500
Epoch 81/250
52000/52000 [==============================] - 310s 6ms/step - loss: 0.5249 - acc: 0.8179 - val_loss: 0.7655 - val_acc: 0.7700

Epoch 00081: val_acc did not improve from 0.79500
Epoch 82/250
52000/52000 [==============================] - 310s 6ms/step - loss: 0.5246 - acc: 0.8166 - val_loss: 0.7717 - val_acc: 0.7706

Epoch 00082: val_acc did not improve from 0.79500
Epoch 83/250
52000/52000 [==============================] - 309s 6ms/step - loss: 0.5162 - acc: 0.8208 - val_loss: 0.7006 - val_acc: 0.7880

Epoch 00083: val_acc did not improve from 0.

52000/52000 [==============================] - 308s 6ms/step - loss: 0.3493 - acc: 0.8785 - val_loss: 0.7009 - val_acc: 0.8116

Epoch 00162: val_acc did not improve from 0.82780
Epoch 163/250
52000/52000 [==============================] - 307s 6ms/step - loss: 0.3417 - acc: 0.8804 - val_loss: 0.6764 - val_acc: 0.8126

Epoch 00163: val_acc did not improve from 0.82780
Epoch 164/250
52000/52000 [==============================] - 306s 6ms/step - loss: 0.3418 - acc: 0.8824 - val_loss: 0.6696 - val_acc: 0.8149

Epoch 00164: val_acc did not improve from 0.82780
Epoch 165/250
52000/52000 [==============================] - 308s 6ms/step - loss: 0.3357 - acc: 0.8848 - val_loss: 0.7290 - val_acc: 0.8064

Epoch 00165: val_acc did not improve from 0.82780
Epoch 166/250
52000/52000 [==============================] - 307s 6ms/step - loss: 0.3355 - acc: 0.8829 - val_loss: 0.7051 - val_acc: 0.8033

Epoch 00166: val_acc did not improve from 0.82780
Epoch 167/250
52000/52000 [===========================

52000/52000 [==============================] - 1430s 28ms/step - loss: 0.2879 - acc: 0.8993 - val_loss: 0.7310 - val_acc: 0.8137

Epoch 00204: val_acc did not improve from 0.83240
Epoch 205/250
52000/52000 [==============================] - 1430s 27ms/step - loss: 0.2863 - acc: 0.8983 - val_loss: 0.6501 - val_acc: 0.8219

Epoch 00205: val_acc did not improve from 0.83240
Epoch 206/250
52000/52000 [==============================] - 1432s 28ms/step - loss: 0.2883 - acc: 0.8983 - val_loss: 0.8418 - val_acc: 0.7923

Epoch 00206: val_acc did not improve from 0.83240
Epoch 207/250
52000/52000 [==============================] - 1414s 27ms/step - loss: 0.2894 - acc: 0.8981 - val_loss: 0.6856 - val_acc: 0.8205

Epoch 00207: val_acc did not improve from 0.83240
Epoch 208/250
52000/52000 [==============================] - 1408s 27ms/step - loss: 0.2851 - acc: 0.9014 - val_loss: 0.6746 - val_acc: 0.8250

Epoch 00208: val_acc did not improve from 0.83240
Epoch 209/250
52000/52000 [=================


Epoch 00244: val_acc did not improve from 0.84530
Epoch 245/250
52000/52000 [==============================] - 1401s 27ms/step - loss: 0.2518 - acc: 0.9121 - val_loss: 0.6262 - val_acc: 0.8423

Epoch 00245: val_acc did not improve from 0.84530
Epoch 246/250
52000/52000 [==============================] - 1403s 27ms/step - loss: 0.2515 - acc: 0.9132 - val_loss: 0.7264 - val_acc: 0.8225

Epoch 00246: val_acc did not improve from 0.84530
Epoch 247/250
52000/52000 [==============================] - 1403s 27ms/step - loss: 0.2509 - acc: 0.9113 - val_loss: 0.7642 - val_acc: 0.8143

Epoch 00247: val_acc did not improve from 0.84530
Epoch 248/250
52000/52000 [==============================] - 1409s 27ms/step - loss: 0.2499 - acc: 0.9127 - val_loss: 0.6000 - val_acc: 0.8400

Epoch 00248: val_acc did not improve from 0.84530
Epoch 249/250
52000/52000 [==============================] - 1403s 27ms/step - loss: 0.2509 - acc: 0.9118 - val_loss: 0.5953 - val_acc: 0.8439

Epoch 00249: val_acc did not 

In [16]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 64s 6ms/step
Test loss: 0.5730056409597397
Test accuracy: 0.8451


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


## Conclusion:

On the CIFAR image dataset using Keras based Convolution Neural Network I have achieved testing accuracy of 84.51%